<a href="https://colab.research.google.com/github/GabrielDan92/PySpark/blob/main/pyspark_tc(2).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org//dist/spark/spark-3.1.2/spark-3.1.2-bin-hadoop3.2.tgz
!tar xf spark-3.1.2-bin-hadoop3.2.tgz
!pip install -q pyspark
!pip install -q findspark

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop3.2"

import findspark
from datetime import datetime
from pyspark.sql import SparkSession
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number, lit, col, array, round, size, when, concat, concat_ws, array_except

findspark.init()
spark = SparkSession.builder \
    .master("local[*]") \
    .appName("spark_TC") \
    .getOrCreate()

     |████████████████████████████████| 212.4 MB 61 kB/s 
     |████████████████████████████████| 198 kB 49.2 MB/s 


In [6]:
stations = [(1, 0, "BAutogara"), \
            (2, 1, "BVAutogara"), \
            (3, 2, "SBAutogara"), \
            (4, 3, "CJAutogara"), \
            (5, 4, "MMAutogara"), \
            (6, 5, "ISAutogara"), \
            (7, 6, "CTAutogara"), \
            (8, 7, "TMAutogara"), \
            (9, 8, "BCAutogara"), \
            (10, 9, "MSAutogara")]

stationsColumns = ["row_num", "internal_bus_station_id", "public_bus_station"]
stationsDF = spark.createDataFrame(data=stations, schema=stationsColumns)

trips = [(1, "B", "MM", [0,2,4], [datetime(2020, 3, 1, 10, 10, 00), datetime(2020, 3, 1, 12, 20, 10), datetime(2020, 3, 1, 14, 10, 10)]), \
        (2, "BV", "IS", [1,8,5,3], [datetime(2020, 3, 1, 8, 10, 00), datetime(2020, 3, 1, 12, 20, 10), datetime(2020, 3, 1, 15, 10, 10), datetime(2020, 3, 1, 15, 45, 10)]), \
        (3, "TM", "CT", [7,2,6,4,9], [datetime(2020, 4, 1, 10, 45, 00), datetime(2020, 4, 1, 12, 20, 10), datetime(2020, 4, 1, 19, 30, 10), datetime(2020, 4, 1, 21, 30, 10), datetime(2020, 4, 1, 22, 00, 10)]), \
        (4, "CJ", "BC", [3,9,5,6,7,1], [datetime(2020, 5, 1, 7, 10, 00), datetime(2020, 5, 1, 12, 20, 10), datetime(2020, 5, 1, 13, 20, 10), datetime(2020, 5, 1, 14, 20, 10), datetime(2020, 5, 1, 15, 20, 10), datetime(2020, 5, 1, 21, 20, 10)])]

tripsColumns = ["row_num", "origin", "destination", "internal_bus_stations_ids", "triptimes"]
tripsDF = spark.createDataFrame(data=trips, schema=tripsColumns)

print("Stations data set:")
stationsDF.show()
tripsDF = tripsDF.withColumn("unique_key", concat_ws("", col("internal_bus_stations_ids")))
print("Trips data set:")
tripsDF.show(truncate=False)
stations = stationsDF.alias("stations")
trips = tripsDF.alias("trips")

print("Get the number of trips (array length):")
columns = trips.select(trips.internal_bus_stations_ids, size("internal_bus_stations_ids").alias("size"))
columns.show()
max = columns.agg({"size": "max"}).collect()[0]
maxArrLength = max["max(size)"]

queryString = ""
tempViewName = "trips"
trips.createTempView(tempViewName)

for i in range(maxArrLength):
    queryString += f"{tempViewName}.internal_bus_stations_ids[{str(i)}] as column_{i+1}"
    if i != maxArrLength - 1:
        queryString += ", "

print("Programatically build a dynamic SQL query to get each array element in its own column.")
print(f"Dynamic SQL query: \n <SELECT {queryString} FROM {tempViewName}>")

tripsNameDF = spark.sql(f"SELECT {queryString} FROM {tempViewName}")
tripsNameDF.show()

print("Use the newly created columns for left joins against the original stations data set.")
columnNames = tripsNameDF.schema.names
joinedName = ""

for name in columnNames:
    joinedName = name + "_public"
    tripsNameDF = tripsNameDF.join(stations, tripsNameDF[name] == stations.internal_bus_station_id, how="left") \
            .select(tripsNameDF["*"], stations["public_bus_station"].alias(joinedName))

tripsNameDF = tripsNameDF.na.fill("")
tripsNameDF.show()

print("Add the individual columns back to the array datatype.")
print("Create a unique_key column - the arrays just created will have 'null' values that will prevent the join from working as expected")
columnNames = tripsNameDF.schema.names
internal = []
public = []

for name in columnNames:
    if name.find("public") != -1:
        public.append(name)
    else:
        internal.append(name)

tripsNameDF = tripsNameDF.select(array(internal).alias("internal_bus_stations"), \
                                array_except(array(public), array(lit(""))).alias("public_bus_stops"))

tripsNameDF = tripsNameDF.withColumn("unique_key_public_stops", concat_ws("", col("internal_bus_stations")))

tripsNameDF.show(truncate=False)

print("Knowing the numbers of trip, capture the longest trip in a variable - it will be used for building the dynamic SQL query")
columns = trips.select(trips.internal_bus_stations_ids.alias("internal_bus_stations"), trips.triptimes, size("triptimes").alias("size"))
columns.show(truncate=False)
max = columns.agg({"size": "max"}).collect()[0]
maxArrLength = max["max(size)"]

print("Use the number of trips to build the dynamic SQL query, creating individual columns for each trip timestamp.")
tempViewName = "triptimes"
queryString = tempViewName + ".internal_bus_stations_ids as internal_bus_stations, "
trips.createTempView(tempViewName)

for i in range(maxArrLength):
    queryString += f"{tempViewName}.triptimes[{str(i)}] as column_{i+1}"
    if i != maxArrLength - 1:
        queryString += ", "

print(f"Dynamic SQL query: \n <SELECT {queryString} FROM {tempViewName}>")

tripsDurationDF = spark.sql(f"SELECT {queryString} FROM {tempViewName}")
tripsDurationDF.show()

print("Calculate the longest trip duration and create the 'duration' column - useful in the next for the 'when' clause.")
columnNames = tripsDurationDF.schema.names
maxIndex = len(columnNames)-1

tripsDurationDF = tripsDurationDF.withColumn("duration", \
                                             col(columnNames[len(columnNames)-1]).cast("int") - \
                                             col(columnNames[1]).cast("int"))
tripsDurationDF.show()

print("Within a reverse for loop, calculate the trip duration for each trip, regardless of how many stops it has.")
print("Convert the duration from seconds to minutes.")
for i in range(maxIndex, 0, -1):
    tripsDurationDF = tripsDurationDF.withColumn("duration", \
                                            when(tripsDurationDF["duration"].isNull(), \
                                            col(columnNames[i]).cast("int") - col(columnNames[1]).cast("int")) \
                                            .otherwise(tripsDurationDF["duration"]))

tripsDurationDF = tripsDurationDF.withColumn("duration", round(tripsDurationDF["duration"])/60)
tripsDurationDF = tripsDurationDF.withColumn("duration", round(tripsDurationDF["duration"],2))
tripsDurationDF = tripsDurationDF.withColumn("duration", concat(col("duration"), lit(" min")))
tripsDurationDF.show(truncate=False)

print("Get the individual timestamps columns and merge them into one array datatype column.")

columnNames = tripsDurationDF.schema.names
timestamps = []

for i in range(1, len(columnNames)-1):
        timestamps.append(columnNames[i])

tripsDurationDF = tripsDurationDF.select(tripsDurationDF.internal_bus_stations, \
                                array(timestamps).alias("triptimes"), tripsDurationDF.duration)

tripsDurationDF.show(truncate=False)

print("Get the duration column and join it in the original trips dataset.")

trips = trips.join(tripsDurationDF, trips.internal_bus_stations_ids == tripsDurationDF.internal_bus_stations) \
        .select(trips["row_num"], trips["unique_key"], trips["internal_bus_stations_ids"], trips["origin"], trips["destination"], tripsDurationDF["duration"])
trips.orderBy(["row_num"]).show(truncate=False)

print("Get the public_stops_column and join it to the trips dataset.")
trips = trips.join(tripsNameDF, trips.unique_key == tripsNameDF.unique_key_public_stops) \
        .select(trips["row_num"], trips["origin"], trips["destination"], trips["internal_bus_stations_ids"], tripsNameDF["public_bus_stops"], trips["duration"])
trips.orderBy(["row_num"]).show(truncate=False)

Stations data set:
+-------+-----------------------+------------------+
|row_num|internal_bus_station_id|public_bus_station|
+-------+-----------------------+------------------+
|      1|                      0|         BAutogara|
|      2|                      1|        BVAutogara|
|      3|                      2|        SBAutogara|
|      4|                      3|        CJAutogara|
|      5|                      4|        MMAutogara|
|      6|                      5|        ISAutogara|
|      7|                      6|        CTAutogara|
|      8|                      7|        TMAutogara|
|      9|                      8|        BCAutogara|
|     10|                      9|        MSAutogara|
+-------+-----------------------+------------------+

Trips data set:
+-------+------+-----------+-------------------------+------------------------------------------------------------------------------------------------------------------------------+----------+
|row_num|origin|destination|i